In [31]:
import json
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import pad_sequences
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.layers import Masking
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
features_d1 = []
labels_d1 = []
# Open file for reading
with open('domain1_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d1.append(obj['text'])
        labels_d1.append(obj['label'])
features_d2 = []
labels_d2 = []
# Open file for reading
with open('domain2_train.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        features_d2.append(obj['text'])
        labels_d2.append(obj['label'])
features = features_d1 + features_d2
labels = labels_d1 + labels_d2

In [32]:

features = pad_sequences(features, padding='pre', maxlen=180, value=-1)
labels = to_categorical(labels, num_classes=2)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [4]:
len(X_train)

27520

In [35]:
vocab_size = 5000  # Adjust based on your actual vocabulary size
embedding_dim = 128
sequence_length = 180
n_classes = 2  # Binary classification

from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Masking, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Flatten, AdditiveAttention, GlobalAveragePooling1D
from keras.regularizers import l1, l2
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Include early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    Masking(mask_value=-1),
    Conv1D(32, 3, activation='relu', kernel_regularizer=l2(0.001)),  # Reduced filter size
    BatchNormalization(),
    MaxPooling1D(4),  # Reduced pooling size
    Dropout(0.5),  # Adjusted dropout
    Bidirectional(LSTM(16, recurrent_regularizer=l2(0.001), kernel_regularizer=l2(0.001))),  # Reduced LSTM units
    Dropout(0.5),  # Adjusted dropout
    Dense(n_classes, activation='sigmoid', kernel_regularizer=l1(0.001))
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [36]:
model.fit(X_train, y_train, validation_split=0.2, epochs=50, callbacks=[early_stopping])

Epoch 1/50
688/688 [==============================] - 18s 20ms/step - loss: 0.4933 - accuracy: 0.8627 - val_loss: 0.3725 - val_accuracy: 0.8999
Epoch 2/50
688/688 [==============================] - 12s 17ms/step - loss: 0.3490 - accuracy: 0.9051 - val_loss: 0.3419 - val_accuracy: 0.9010
Epoch 3/50
688/688 [==============================] - 11s 16ms/step - loss: 0.3050 - accuracy: 0.9118 - val_loss: 0.3385 - val_accuracy: 0.8944
Epoch 4/50
688/688 [==============================] - 11s 16ms/step - loss: 0.2672 - accuracy: 0.9191 - val_loss: 0.3467 - val_accuracy: 0.8997
Epoch 5/50
688/688 [==============================] - 11s 16ms/step - loss: 0.2395 - accuracy: 0.9267 - val_loss: 0.3671 - val_accuracy: 0.8872
Epoch 6/50
688/688 [==============================] - 11s 16ms/step - loss: 0.2206 - accuracy: 0.9344 - val_loss: 0.3652 - val_accuracy: 0.8875


In [50]:
test_ids = []
test_texts = []
# Open file for reading
with open('test_set.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)
        test_ids.append(obj['id'])
        test_texts.append(obj['text'])

In [51]:
test_features = pad_sequences(test_texts, padding='pre', maxlen=180, value=-1)

In [49]:
len(final_predictions)

34400

In [52]:
import numpy as np
# Make predictions
predictions = model.predict(test_features)

# Interpret predictions
final_predictions = np.argmax(predictions, axis=1)

print(final_predictions)

32/32 [==============================] - 1s 8ms/step
[0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 1 1
 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0 0 1 1 1 1
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0
 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0
 0 0 0 0 1 1 1 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1
 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1
 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 

In [53]:
import pandas as pd
output_df = pd.DataFrame({"id":test_ids, "class": final_predictions})

In [55]:
output_df.to_csv("output.csv", index=False)